<a href="https://colab.research.google.com/github/shi093/interpolate/blob/main/Quarterly_to_Monthly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datetime-quarter
!pip install numpy

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import time
import pytz, tzlocal
import string
import plotly.graph_objects as go
from datequarter import DateQuarter
from scipy.interpolate import CubicSpline

In [3]:
# read and parse part of "Summary" worksheet
def get_sum_region(start, size):
    emp = pd.DataFrame()
    for i in range(size):
        df1 = df_sum.iloc[5:26, start+i]
        emp = pd.concat([emp, df1], axis=0)
    
    emp.reset_index(drop = True, inplace = True)
    emp.columns = ['emp']
    
    emp_p = pd.DataFrame()
    for i in range(size):
        df1 = df_sum.iloc[28:49, start+i]
        emp_p = pd.concat([emp_p, df1], axis=0)
    
    emp_p.reset_index(drop = True, inplace = True)
    emp_p.columns = ['emp_percent']

    gdp = pd.DataFrame()
    for i in range(size):
        df1 = df_sum.iloc[53:74, start+i]
        gdp = pd.concat([gdp, df1], axis=0)
    
    gdp.reset_index(drop = True, inplace = True)
    gdp.columns = ['gdp']
    
    gdp_p = pd.DataFrame()
    for i in range(size):
        df1 = df_sum.iloc[76:97, start+i]
        gdp_p = pd.concat([gdp_p, df1], axis=0)
    
    gdp_p.reset_index(drop = True, inplace = True)
    gdp_p.columns = ['gdp_percent']
    
    pop = pd.DataFrame()
    for i in range(size):
        df1 = df_sum.iloc[101:122, start+i]
        pop = pd.concat([pop, df1], axis=0)
    
    pop.reset_index(drop = True, inplace = True)
    pop.columns = ['pop']
    
    pop_p = pd.DataFrame()
    for i in range(size):
        df1 = df_sum.iloc[124:145, start+i]
        pop_p = pd.concat([pop_p, df1], axis=0)
    
    pop_p.reset_index(drop = True, inplace = True)
    pop_p.columns = ['pop_percent']
    
    return(pd.concat([emp, emp_p, gdp, gdp_p, pop, pop_p], axis=1))

# Employment and GDP by sector for the region (df = US, NYPA, NYCY, etc)

def get_sector(start, size, df):
    emp = pd.DataFrame()
    for i in range(size):
        df1 = df.iloc[5:26, start+i]
        emp = pd.concat([emp, df1], axis=0)
    
    emp.reset_index(drop = True, inplace = True)
    emp.columns = ['emp']

    gdp = pd.DataFrame()
    for i in range(size):
        df1 = df.iloc[29:50, start+i]
        gdp = pd.concat([gdp, df1], axis=0)
    
    gdp.reset_index(drop = True, inplace = True)
    gdp.columns = ['gdp']
    
    return(pd.concat([emp, gdp], axis=1))

# Imcome info by region (df = US, NYPA, NYCY, etc)

def get_income(begin, end, df):
    # average income from employment (thousands US$)
    avg_income = pd.DataFrame(df.iloc[52, begin:end])
    avg_income.columns = ['avg_income']
    avg_income.reset_index(drop=True, inplace=True)

    # Personal disposable income (million US$)
    disp_income = pd.DataFrame(df.iloc[53, begin:end])
    disp_income.columns = ['disp_income']
    disp_income.reset_index(drop=True, inplace=True)

    # Real personal disposable income (million US$, constant 2012 prices)
    rdisp_income = pd.DataFrame(df.iloc[54, begin:end])
    rdisp_income.columns = ['rdisp_income']
    rdisp_income.reset_index(drop=True, inplace=True)

    # Retail sales (millions US$)
    retail = pd.DataFrame(df.iloc[55, begin:end])
    retail.columns = ['retail']
    retail.reset_index(drop=True, inplace=True)

    # Real retail sales (millions US$, constant 2012 prices)
    r_retail = pd.DataFrame(df.iloc[56, begin:end])
    r_retail.columns = ['r_retail']
    r_retail.reset_index(drop=True, inplace=True)
    
    return(pd.concat([avg_income, disp_income, rdisp_income, retail, r_retail], axis=1))

In [4]:
# Read in the Oxford County dataset
# create functions to automate the data processing

file_path = '/content/drive/MyDrive/Oxford_2021Q2/County dataset.xlsm'
xls = pd.ExcelFile(file_path)

# to read all sheets to a map
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
    
# Get the "Summary" worksheet
df_sum = sheet_to_df_map['Summary']

# create time and region columns
# Year column
year = pd.DataFrame(np.arange(1980,2036), columns = ['year'])
year.reset_index(drop=True, inplace=True)
# Quarter column, repeat quarter 1-4 for each year
q = pd.DataFrame(np.arange(1, 5), columns = ['quarter'])
q.reset_index(drop=True, inplace=True)
quarter = pd.concat([q]*56, ignore_index=True)
# Year column for quarterly report, i.e., repeat every year 4 times
year_q = pd.DataFrame(np.repeat(year.values,4,axis=0), columns = ['year'])

# Get the 20 regions and US as region_id=0 (total 21 regions)
region = pd.DataFrame(df_sum.iloc[5:26, 0])
region.reset_index(drop = True, inplace = True)
region.columns = ['regions']
region['region_id'] = np.arange(len(region))
# repeat regions for each year: 21*56
regions = pd.concat([region]*56, ignore_index=True)
# repeat year for each region
year_region = pd.DataFrame(np.repeat(year.values,21,axis=0), columns = ['year'])

# repeat 21 region list for each quarter
regions_q = pd.concat([region]*224, ignore_index=True)
# repeat each year 84 times for each region-quarter combination
year_region_q = pd.DataFrame(np.repeat(year.values,84,axis=0), columns = ['year'])
# repeat quarter 1-4 and year combination for each region
quarter_region = pd.DataFrame(np.repeat(quarter.values,21,axis=0), columns = ['quarter'])

# read "Summary", start from column 3, from year 1980 to 2035
df_data = get_sum_region(3, 56)
sum_region = pd.concat([year_region, regions, df_data], axis=1)

# read "Summary", Quarterly data, start from column 60, from year 1980Q1 to 2035Q4
df_data = get_sum_region(60, 224)
sum_region_q = pd.concat([year_region_q, quarter_region, regions_q, df_data], axis=1)

# Get the "US" worksheet
df_us = sheet_to_df_map['US']

# read in the list of sectiors, 21 including "Total" as sector 0
sector = pd.DataFrame(df_us.iloc[5:26, 0])
sector.reset_index(drop = True, inplace = True)
sector.columns = ['sectors']
sector['sector_id'] = np.arange(len(sector))
# sector set repeat for each year
sectors = pd.concat([sector]*56, ignore_index=True)
year_sector = pd.DataFrame(np.repeat(year.values,21,axis=0), columns = ['year'])
# sector set repeat for each year-quarter combination
sectors_q = pd.concat([sector]*224, ignore_index=True)
# repeat each year 84 times for each quarter-sector combination
year_sector_q = pd.DataFrame(np.repeat(year.values,84,axis=0), columns = ['year'])
# repeat each quarter 21 times for each sector
quarter_sector = pd.DataFrame(np.repeat(quarter.values,21,axis=0), columns = ['quarter'])

# "US", "NYPA", "NYCT", "NY01", ... all have the same data structure
region_list = xls.sheet_names

sector_region = pd.DataFrame()
sector_region_q = pd.DataFrame()
income_region = pd.DataFrame()
income_region_q = pd.DataFrame()

for r in region_list[2:23]:
    df_r = sheet_to_df_map[r]
    
    df_data = get_sector(3, 56, df_r)
    df1 = pd.concat([year_sector, sectors, df_data], axis=1)
    df1['region']= r
    sector_region = pd.concat([sector_region, df1], axis=0)

    df_data = get_sector(60, 224, df_r)
    df2 = pd.concat([year_sector_q, quarter_sector, sectors_q, df_data], axis=1)
    df2['region']= r
    sector_region_q = pd.concat([sector_region_q, df2], axis=0)
    
    df_data = get_income(3, 59, df_r)
    df3 = pd.concat([year, df_data], axis=1)
    df3['region'] = r
    income_region = pd.concat([income_region, df3], axis=0)
    
    df_data = get_income(60, 284, df_r)
    df4 = pd.concat([year_q, quarter, df_data], axis=1)
    df4['region'] = r
    income_region_q = pd.concat([income_region_q, df4], axis=0)

In [5]:
## Explore variables

## region and region ID
#region.head()

## list of region worksheet names
#region_list

## sector and sector ID
#sector.head()

sum_region_q.head()    # Summary sheet

## Employment/GDP by sector by region, in US, NYPA NYCY and all the region sheets
#sector_region_q.head() 

## Incomes and Spending by region, in US, NYPA NYCY and all the region sheets
#income_region_q.head()

,year,quarter,regions,region_id,emp,emp_percent,gdp,gdp_percent,pop,pop_percent
0,1980,1,US,0,90892.3,NaN,1709400,NaN,226145,NaN
1,1980,1,Port Authority Area,1,NaN,NaN,NaN,NaN,15779.3,NaN
2,1980,1,New York City,2,NaN,NaN,NaN,NaN,7104.68,NaN
3,1980,1,Bronx County,3,NaN,NaN,NaN,NaN,1174.43,NaN
4,1980,1,Kings County,4,NaN,NaN,NaN,NaN,2241.45,NaN


In [6]:
# functions used in interpolation method

def get_ym(i, start_year):
    year = int(i/12)
    month = int(i%12+1)
    if (month == 0 and year>0):
        year = year -1
        
    year = start_year + year
    ym = str(year)+'-'+str(month).zfill(2)
    return(ym) 

def get_ym_next(i, start_year):
    i = i+1
    year = int(i/12)
    month = int(i%12+1)
    if (month == 0 and year>0):
        year = year -1
        
    year = start_year + year
    ym = str(year)+'-'+str(month).zfill(2)
    return(ym) 

def interpolate_sum(df, var, start_year):
    series2 = df.loc[:, ['q_date', var]]
    series2.columns = ['datetime', var]
    series2[var] = pd.to_numeric(series2[var])

    series2.loc[-1] = [str(start_year-1)+"-12-31", 0]  # adding a row
    series2.index = series2.index + 1  # shifting index
    series2 = series2.sort_index()  # sorting by index

    series2['datetime'] = pd.to_datetime(series2['datetime'])

    # variable value for each quarter is incremental values for each quarter
    # Generate cumulative series
    series2[var] = series2[var].cumsum()
    x = []
    for i in range(len(series2.datetime)):
        x.append((series2.datetime[i].year-start_year)*12 + series2.datetime[i].month)
    y = np.array(series2[var], dtype = 'float')
    f = CubicSpline(x, y, bc_type='natural')

    num_years = 2035 - start_year + 1
    x_new = np.linspace(0, num_years*12, num_years*12+1)
    y_new = f(x_new)

    x_ym = []
    for i in range(len(x_new)):
        x_ym.append(get_ym(x_new[i], start_year))

    output_variable = pd.concat([pd.DataFrame(x_new), pd.DataFrame(x_ym), pd.DataFrame(y_new)], axis = 1)
    output_variable.columns = ['timeid', 'datetime',var]
    output_variable['datetime'] = pd.to_datetime(output_variable['datetime'])
    output_variable['datetime'] = output_variable['datetime'] - dt.timedelta(days = 1)
    output_variable[var] = output_variable[var].diff()
    output_variable = output_variable.loc[output_variable["timeid"] > 0, ['datetime', var]]
    return(output_variable)

def interpolate_average(df, var, start_year, factor):

    series3 = df.loc[:, ['month_id', var]]
    series3.columns = ['datetime', var]
    series3[var] = pd.to_numeric(series3[var]*3/factor)
    
    series3.loc[-1] = [0, 0]  # adding a row
    series3.index = series3.index + 1  # shifting index
    series3= series3.sort_index()  # sorting by index
    
    series3[var] = series3[var].cumsum()

    x = []
    x = np.array(series3['datetime'], dtype = 'int')
    y = np.array(series3[var], dtype = 'float')
    f = CubicSpline(x, y, bc_type='natural')
    
    num_years = 2035 - start_year + 1
    x_new = np.linspace(0, num_years*12, num_years*12+1)
    y_dev = f(x_new, 1)*factor
    
    x_datetime = []
    for i in range(len(x_new)):
        x_datetime.append(get_ym_next(x_new[i], start_year))

    monthly_avg = pd.concat([pd.DataFrame(x_new), pd.DataFrame(x_datetime), pd.DataFrame(y_dev)], axis = 1)
    monthly_avg.columns = ['time_id','datetime', var]
    monthly_avg['datetime'] = pd.to_datetime(monthly_avg['datetime'])
    monthly_avg['datetime'] = monthly_avg['datetime'] - dt.timedelta(days = 1)
    return(monthly_avg)

def monthly_average(df, var, start_year, factor):

    series3 = df.loc[:, ['month_id', var]]
    series3.columns = ['datetime', var]
    series3[var] = pd.to_numeric(series3[var]*3/factor)
    
    series3.loc[-1] = [0, 0]  # adding a row
    series3.index = series3.index + 1  # shifting index
    series3= series3.sort_index()  # sorting by index
    
    series3[var] = series3[var].cumsum()

    x = []
    x = np.array(series3['datetime'], dtype = 'int')
    y = np.array(series3[var], dtype = 'float')
    f = CubicSpline(x, y, bc_type='natural')
    
    num_years = 2035 - start_year + 1
    x_new = np.linspace(0, num_years*12, num_years*12+1)
    x_datetime = []  
    y_month = []
    for m in range(len(x_new)):
        y_month.append(np.mean(f(np.linspace(m, m+1, 1001), 1))*factor)
        x_datetime.append(get_ym_next(x_new[m], start_year))
    
    monthly_avg = pd.concat([pd.DataFrame(x_new), pd.DataFrame(x_datetime), pd.DataFrame(y_month)], axis = 1)
    monthly_avg.columns = ['time_id','datetime', var]
    monthly_avg['datetime'] = pd.to_datetime(monthly_avg['datetime'])
    monthly_avg['datetime'] = monthly_avg['datetime'] - dt.timedelta(days = 1)

    return(monthly_avg)

In [7]:
# GDP by region

# the year the data became available
start_year = 2001
# region ID
region_ID = 2
# select variable for interpolation
var = "gdp"  

df2 = sum_region_q.loc[(sum_region_q["region_id"] == region_ID) & (sum_region_q["year"]>=start_year), ["year", "quarter", "gdp", "region_id"]]
df2.reset_index(drop = True, inplace = True)
end_date = []
for i in range(len(df2)):
    end_date.append(DateQuarter(df2.iloc[i].year, df2.iloc[i].quarter).end_date())
df2['q_date'] = pd.DataFrame(end_date)

output_variable = interpolate_sum(df2, var, start_year )
output_variable.head()

,datetime,gdp
1,2001-01-31,53003.516123
2,2001-02-28,53091.537290
3,2001-03-31,53267.579623
4,2001-04-30,53502.299907
5,2001-05-31,53678.325274


In [8]:
# GDP by region by sector: US, NYPA, NYCY and other regions worksheets

# the year the data became available
start_year = 1980
# region worksheet name
region_worksheet = 'US'
# select sector
sector_ID = 1
# select the varialbe to be interpolated
var = "gdp"

# select the dataset
input_variable = sector_region_q 

df2 = input_variable.loc[(input_variable["region"] == region_worksheet) & 
                         (input_variable["year"]>=start_year) & 
                         (input_variable["sector_id"]==sector_ID), 
                         ["year", "quarter", var]]
df2.reset_index(drop = True, inplace = True)
end_date = []
for i in range(len(df2)):
    end_date.append(DateQuarter(df2.iloc[i].year, df2.iloc[i].quarter).end_date())
df2['q_date'] = pd.DataFrame(end_date)

output_variable = interpolate_sum(df2, var, start_year)
output_variable.head()

,datetime,gdp
1,1980-01-31,21033.820413
2,1980-02-29,21070.384738
3,1980-03-31,21143.513389
4,1980-04-30,21239.545587
5,1980-05-31,21303.838222


In [9]:
# Incomes and Spending
# By region by sector: US, NYPA, NYCY and other regions worksheets

# the year the data became available
start_year = 1992
# region worksheet name
region_worksheet = 'US'

# select the dataset
input_variable = income_region_q
# select variable: avg_income	disp_income	rdisp_income	retail	r_retail	
var = "retail"

df2 = input_variable.loc[(input_variable["region"] == region_worksheet) & 
                         (input_variable["year"]>=start_year), 
                         ["year", "quarter", var]]
df2.reset_index(drop = True, inplace = True)
end_date = []
for i in range(len(df2)):
    end_date.append(DateQuarter(df2.iloc[i].year, df2.iloc[i].quarter).end_date())
df2['q_date'] = pd.DataFrame(end_date)

output_variable = interpolate_sum(df2, var, start_year)
output_variable.head()

,datetime,retail
1,1992-01-31,153191.067883
2,1992-02-29,153443.516971
3,1992-03-31,153948.415147
4,1992-04-30,154668.720730
5,1992-05-31,155456.266999


In [10]:
# using interpolate_average & monthly_average function
# Select the variable: emp or pop 

var = "emp"

region_ID = 0
# the year the data became available
if region_ID == 0:
    start_year = 1980
else:
    start_year = 1990

df3 = sum_region_q.loc[(sum_region_q["region_id"] == region_ID) & 
                        (sum_region_q["year"]>=start_year),
                        ["year", "quarter", var]]
df3.reset_index(drop = True, inplace = True)
end_date = []
month_id = []

for i in range(len(df3)):
    month_id.append((df3.iloc[i].year-start_year)*12+(df3.iloc[i].quarter)*3)
    end_date.append(DateQuarter(df3.iloc[i].year, df3.iloc[i].quarter).end_date())
df3['month_id'] = pd.DataFrame(month_id)
df3['datetime'] = pd.DataFrame(end_date)
df3['datetime'] = pd.to_datetime(df3['datetime'])

factor = 10000
interpolated_monthly = monthly_average(df3, var, start_year, factor)
interpolated_quarterly = interpolate_average(df3, var, start_year, factor)
    
print(interpolated_monthly.head())
print(interpolated_quarterly.head())

   time_id   datetime           emp
0      0.0 1980-01-31  90960.208540
1      1.0 1980-02-29  90909.303954
2      2.0 1980-03-31  90807.494780
3      3.0 1980-04-30  90657.277965
4      4.0 1980-05-31  90468.633806
   time_id   datetime           emp
0      0.0 1980-01-31  90968.696880
1      1.0 1980-02-29  90943.244587
2      2.0 1980-03-31  90866.887707
3      3.0 1980-04-30  90739.626240
4      4.0 1980-05-31  90568.947280


In [11]:
# Plotting the results

# remove scrolling output window feature
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

import plotly.graph_objects as go

# set report variable
report_variable = "Employment"
# set plot title
plot_title = "Monthly "+report_variable

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=interpolated_quarterly.time_id[0:72], y=interpolated_quarterly[var][0:72],
                    mode='lines', line_shape = 'spline',
                    name='smooth'))
fig.add_trace(go.Scatter(x=interpolated_monthly.time_id[0:72], y=interpolated_monthly[var][0:72],
                    mode='lines', line_shape = 'hv',
                    name='monthly'))
fig.add_trace(go.Scatter(x=df3.month_id[0:24], y=df3[var][0:24],
                    mode='lines', line_shape = 'vh',
                    name='quarterly'))

fig.update_layout(
    title = plot_title,
    xaxis_title="Time",
    yaxis_title=report_variable,
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

fig.show()


In [12]:
# Plotting the results

# remove scrolling output window feature
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

import plotly.graph_objects as go

# set report variable
report_variable = "Employment"
# set plot title
plot_title = region.iloc[region_ID].regions + " Monthly "+ report_variable

fig = go.Figure()

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=interpolated_quarterly.time_id, y=interpolated_quarterly.emp,
                    mode='lines', line_shape = 'spline',
                    name='smooth'))
fig.add_trace(go.Scatter(x=interpolated_monthly.time_id, y=interpolated_monthly.emp,
                    mode='lines', line_shape = 'hv',
                    name='monthly'))
fig.add_trace(go.Scatter(x=df3.month_id, y=df3.emp,
                    mode='lines', line_shape = 'vh',
                    name='quarterly'))

fig.update_layout(
    title = plot_title,
    xaxis_title="Time",
    yaxis_title=report_variable,
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

fig.show()

In [13]:
def interpolate_average2(df, var, start_year, end_year, factor):

    series3 = df.loc[:, ['month_id', var]]
    series3.columns = ['datetime', var]
    series3[var] = pd.to_numeric(series3[var]*3/factor)
    
    series3.loc[-1] = [0, 0]  # adding a row
    series3.index = series3.index + 1  # shifting index
    series3= series3.sort_index()  # sorting by index
    
    series3[var] = series3[var].cumsum()

    x = []
    x = np.array(series3['datetime'], dtype = 'int')
    y = np.array(series3[var], dtype = 'float')
    f = CubicSpline(x, y, bc_type='natural')
    
    num_years = end_year - start_year + 1
    x_new = np.linspace(0, num_years*12, num_years*12+1)
    y_dev = f(x_new, 1)*factor
    
    x_datetime = []
    for i in range(len(x_new)):
        x_datetime.append(get_ym_next(x_new[i], start_year))

    monthly_avg = pd.concat([pd.DataFrame(x_new), pd.DataFrame(x_datetime), pd.DataFrame(y_dev)], axis = 1)
    monthly_avg.columns = ['time_id','datetime', var]
    monthly_avg['datetime'] = pd.to_datetime(monthly_avg['datetime'])
    monthly_avg['datetime'] = monthly_avg['datetime'] - dt.timedelta(days = 1)
    return(monthly_avg)

def monthly_average2(df, var, start_year, end_year, factor):

    series3 = df.loc[:, ['month_id', var]]
    series3.columns = ['datetime', var]
    series3[var] = pd.to_numeric(series3[var]*3/factor)
    
    series3.loc[-1] = [0, 0]  # adding a row
    series3.index = series3.index + 1  # shifting index
    series3= series3.sort_index()  # sorting by index
    
    series3[var] = series3[var].cumsum()

    x = []
    x = np.array(series3['datetime'], dtype = 'int')
    y = np.array(series3[var], dtype = 'float')
    f = CubicSpline(x, y, bc_type='natural')
    
    num_years = end_year - start_year + 1
    x_new = np.linspace(0, num_years*12, num_years*12+1)
    x_datetime = []  
    y_month = []
    for m in range(len(x_new)):
        y_month.append(np.mean(f(np.linspace(m, m+1, 1001), 1))*factor)
        x_datetime.append(get_ym_next(x_new[m], start_year))
    
    monthly_avg = pd.concat([pd.DataFrame(x_new), pd.DataFrame(x_datetime), pd.DataFrame(y_month)], axis = 1)
    monthly_avg.columns = ['time_id','datetime', var]
    monthly_avg['datetime'] = pd.to_datetime(monthly_avg['datetime'])
    monthly_avg['datetime'] = monthly_avg['datetime'] - dt.timedelta(days = 1)

    return(monthly_avg)

In [14]:
file_path = '/content/drive/MyDrive/Interpolate/BLS_employment.xlsx'
xls = pd.ExcelFile(file_path)

# to read all sheets to a map
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
    
# Get the "Summary" worksheet
df_quarter = sheet_to_df_map['Quarterly']
df_quarter.reset_index(drop=True, inplace=True)

df_month = sheet_to_df_map['Monthly']
df_month.reset_index(drop=True, inplace=True)

year = pd.DataFrame(np.arange(1990,2021), columns = ['year'])
year.reset_index(drop=True, inplace=True)
# Quarter column, repeat quarter 1-4 for each year
q = pd.DataFrame(np.arange(1, 5), columns = ['quarter'])
q.reset_index(drop=True, inplace=True)
quarter = pd.concat([q]*31, ignore_index=True)
# Year column for quarterly report, i.e., repeat every year 4 times
year_q = pd.DataFrame(np.repeat(year.values,4,axis=0), columns = ['year'])
year_q.reset_index(drop=True, inplace=True)

# read Quarterly data

quarterly_us = pd.concat([year_q, quarter, df_quarter['US'][0:124]], axis=1)
quarterly_us.columns = ['year', 'quarter', 'US']
quarterly_us.reset_index(drop=True, inplace=True)
quarterly_us = quarterly_us.astype({"year": int, "quarter":int, "US":object}).copy()

end_date = []
month_id = []
start_year = 1990
end_year = 2020

for i in range(len(quarterly_us)):
    month_id.append((quarterly_us.iloc[i].year-start_year)*12+(quarterly_us.iloc[i].quarter)*3)
    end_date.append(DateQuarter(quarterly_us.iloc[i].year, quarterly_us.iloc[i].quarter).end_date())
quarterly_us['month_id'] = pd.DataFrame(month_id)
quarterly_us['datetime'] = pd.DataFrame(end_date)
quarterly_us['datetime'] = pd.to_datetime(quarterly_us['datetime'])

factor = 10000
var = "US"
interpolated_monthly = monthly_average2(quarterly_us, var, start_year, end_year, factor)
interpolated_quarterly = interpolate_average2(quarterly_us, var, start_year, end_year, factor)
    
print(interpolated_monthly.head())
print(interpolated_quarterly.head())

   time_id   datetime             US
0      0.0 1990-01-31  109334.802714
1      1.0 1990-02-28  109403.204954
2      2.0 1990-03-31  109540.009433
3      3.0 1990-04-30  109717.651988
4      4.0 1990-05-31  109825.958620
   time_id   datetime             US
0      0.0 1990-01-31  109323.396641
1      1.0 1990-02-28  109357.597760
2      2.0 1990-03-31  109460.201120
3      3.0 1990-04-30  109631.206719
4      4.0 1990-05-31  109787.963394


In [15]:
# Plotting the results

# remove scrolling output window feature
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

import plotly.graph_objects as go

# set report variable
report_variable = "Employment"
# set plot title
plot_title = " Monthly "+ report_variable

fig = go.Figure()

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=interpolated_quarterly.time_id, y=interpolated_quarterly["US"],
                    mode='lines', line_shape = 'spline',
                    name='smooth'))
fig.add_trace(go.Scatter(x=interpolated_monthly.time_id, y=interpolated_monthly["US"],
                    mode='lines', line_shape = 'hv',
                    name='monthly'))
fig.add_trace(go.Scatter(x=quarterly_us.month_id, y=quarterly_us["US"],
                    mode='lines', line_shape = 'vh',
                    name='quarterly'))

fig.update_layout(
    title = plot_title,
    xaxis_title="Time",
    yaxis_title=report_variable,
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

fig.show()


In [16]:
# Plotting the results

# remove scrolling output window feature
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

import plotly.graph_objects as go

# set report variable
report_variable = "Employment"
# set plot title
plot_title = " Monthly "+ report_variable

fig = go.Figure()

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=interpolated_monthly.time_id, y=df_month["US"],
                    mode='lines', line_shape = 'spline',
                    name='true monthly'))
fig.add_trace(go.Scatter(x=interpolated_monthly.time_id, y=interpolated_monthly["US"],
                    mode='lines', line_shape = 'spline',
                    name='estimated monthly'))

fig.update_layout(
    title = plot_title,
    xaxis_title="Time",
    yaxis_title=report_variable,
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

fig.show()


In [18]:
from google.colab import files

#output_file_name = input('Output File Name: ')
#interpolated_monthly.to_csv(output_file_name+'.csv') 
#files.download(output_file_name+'.csv')